In [1]:
import glob
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
from skimage.filters import threshold_otsu
try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET

In [2]:
def non_max_suppression_fast(boxes, overlapThresh):
   # if there are no boxes, return an empty list
   if len(boxes) == 0:
      return []
 
   # initialize the list of picked indexes   
   pick = []

   # grab the coordinates of the bounding boxes
   x = boxes[:,0].astype("float")
   y = boxes[:,1].astype("float")
   w = boxes[:,2].astype("float")
   h = boxes[:,3].astype("float")

   x1 = x - w/2
   y1 = y - h/2
   x2 = x + w/2
   y2 = y + h/2

   # compute the area of the bounding boxes and sort the bounding
   # boxes by the bottom-right y-coordinate of the bounding box
   area = (w + 1) * (h + 1)
   idxs = np.argsort(y2)

   # keep looping while some indexes still remain in the indexes
   # list
   while len(idxs) > 0:
      # grab the last index in the indexes list and add the
      # index value to the list of picked indexes
      last = len(idxs) - 1
      i = idxs[last]
      pick.append(i)

      # find the largest (x, y) coordinates for the start of
      # the bounding box and the smallest (x, y) coordinates
      # for the end of the bounding box
      xx1 = np.maximum(x1[i], x1[idxs[:last]])
      yy1 = np.maximum(y1[i], y1[idxs[:last]])
      xx2 = np.minimum(x2[i], x2[idxs[:last]])
      yy2 = np.minimum(y2[i], y2[idxs[:last]])

      # compute the width and height of the bounding box
      w = np.maximum(0, xx2 - xx1 + 1)
      h = np.maximum(0, yy2 - yy1 + 1)

      # compute the ratio of overlap
      overlap = (w * h) / area[idxs[:last]]

      # delete all indexes from the index list that have
      idxs = np.delete(idxs, np.concatenate(([last],
         np.where(overlap > overlapThresh)[0])))

   # return only the bounding boxes that were picked using the
   # integer data type
   return boxes[pick].astype("int")

In [3]:
with open("VOCNACD/ImageSets/Main/train.txt") as f:
    filenames = f.readlines()
filenames = [x.strip() for x in filenames]

for filename in filenames:
    annotation_path = os.path.join('VOCNACD', 'Annotations',filename+'.xml')
    image_path = os.path.join('VOCNACD', 'JPEGImages',filename+'.jpg')
    img = cv2.imread(image_path)
    os.makedirs('clusters_training/'+filename)
    root = ET.parse(annotation_path).getroot()
    i = 0
    for obj in root.iter('object'):
        xml_box = obj.find('bndbox')
        xmin = int(xml_box.find('xmin').text)
        ymin = int(xml_box.find('ymin').text)
        xmax = int(xml_box.find('xmax').text)
        ymax = int(xml_box.find('ymax').text)
        crop = img[ymin:ymax,xmin:xmax]
        crop_path = os.path.join('clusters_training',filename,filename+'_'+str(i).zfill(3)+'.jpg')
        cv2.imwrite(crop_path,crop)
        i = i + 1